In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib.dates import DayLocator, DateFormatter
import matplotlib.dates as mdates

from mpl_toolkits.axes_grid1 import host_subplot

%matplotlib widget

PCD_DIR = "res/point_clouds_meshed"
PCD_PATTERN = "sampled*.ply"
OUT_DIR = "res/volumes_variations"
DOD_DIR = "x"
TSTEP = 5
GRID_STEP = 0.3
TEMPERATURE_FILE = "res/temperature_data/temperature_zamboni.csv"

icepy_dir = Path.cwd().parents[1]

In [ ]:
# Paths
pcd_dir = icepy_dir / Path(PCD_DIR)
out_dir = icepy_dir / Path(OUT_DIR)
temp_path = icepy_dir / Path(TEMPERATURE_FILE)

# Output file
fout_name = (
    f"{PCD_PATTERN.split('*')[0]}_dir{DOD_DIR.upper()}_tstep{TSTEP}_grid{GRID_STEP}"
)
fout = out_dir / f"{fout_name}.csv"

# import matplotlib.dates as mdates
dateFmt = DateFormatter('%b')

# Read volume results from file
column_names = [
    "pcd0",
    "pcd1",
    "volume",
    "addedVolume",
    "removedVolume",
    "surface",
    "matchingPercent",
    "averageNeighborsPerCell",
]
df = pd.read_csv(fout, sep=",", names=column_names)


# Build date index, sort dataframe and compute dt
max_surface_match = df["matchingPercent"].to_numpy().max()
df["date_in"] = pd.to_datetime(
    df["pcd0"].str.replace(f"{PCD_PATTERN.split('*')[0]}_", ""), format="%Y_%m_%d"
)
df.sort_values(by="date_in", inplace=True)

df["date_fin"] = pd.to_datetime(
    df["pcd1"].str.replace(f"{PCD_PATTERN.split('*')[0]}_", ""), format="%Y_%m_%d"
)
df["dt"] = (df.date_fin - df.date_in) / np.timedelta64(1, "D")

# Compute daily volume variation and normalize by area
df["volume_daily"] = df["volume"] / df["dt"]
df["volume_daily_normalized"] = (
    df["volume_daily"] / df["matchingPercent"] * max_surface_match
)

# Compute cumulated volumes
df["volume_daily_cumul"] = df["volume_daily"].cumsum()
df["volume_daily_norm_cumul"] = df["volume_daily_normalized"].cumsum()


# Read temperature data
temp_df = pd.read_csv(temp_path, sep=",")
temp_df["data"] = pd.to_datetime(temp_df["data"], format="%d/%m/%Y")

start_date = df["date_in"].min()
end_date = df["date_fin"].max()

mask = (temp_df['data'] >= start_date) & (temp_df['data'] <= end_date)
temp_df = temp_df.loc[mask] 

temp_df["Tmed_5d"] = temp_df["Tmed"] .rolling(5, min_periods=1, center=False).mean()

In [ ]:
# Make plot for Daily volumes
fig, ax = plt.subplots()
fig.set_tight_layout(True)
ax.grid(visible=True, which="both")
ax.plot(df["date_in"], -df["volume_daily_normalized"])
ax.set_xlabel("Date")
ax.xaxis.set_major_formatter(dateFmt)
ax.set_ylabel("-dV [$m^3$]")
ax.set_title(f"Daily volume differences - Step {TSTEP} days")
ax.minorticks_on()
ax.grid(which="major", axis="y", linewidth=0.5, color="black")
ax.grid(which="major", axis="x", linewidth=0.3, color="black")
ax.grid(which="minor", axis="y", linestyle=":", linewidth=0.5, color="black")
ax.grid(which="minor", axis="x", linestyle=":", linewidth=0.3, color="black")
ax.xaxis.set_minor_locator(
    DayLocator(bymonthday=[7, 14, 21, 28], interval=1, tz=None)
)
# fig.set_size_inches(18.5, 10.5)
fig.savefig(out_dir / f"{fout_name}_daily_diff_norm.png", dpi=300)



In [ ]:
# Volume vs temperature

# Create host and parasite subplots
fig, host = plt.subplots(figsize=(10, 6))
par = host.twinx()

# Create temperature and volume plots
p1, = host.plot(temp_df["data"], temp_df["Tmed_5d"], label="Temperature", linewidth=1, color="tab:blue")
p2, = par.plot(df["date_in"] + np.timedelta64(5, "D"), -df["volume_daily_normalized"], label="Volume", linewidth=1, color="tab:red")

# Set x and y labels
host.set_xlabel("Date", fontsize=12)
host.set_ylabel("Temperature [$^{\circ}$C]", fontsize=12, color="tab:blue")
par.set_ylabel("Cumulative volume loss [$m^3$]", fontsize=12, color="tab:red")

# Set x-axis date format
dateFmt = DateFormatter('%b')
host.xaxis.set_major_formatter(dateFmt)

# Set grid lines
host.grid(which="both", linestyle="--", alpha=0.5)
host.grid(which="major", axis="y", linewidth=0.5, color="black")
host.grid(which="major", axis="x", linewidth=0.5, color="black")
host.xaxis.set_minor_locator(DayLocator(bymonthday=[7, 14, 21, 28], interval=1, tz=None))
host.grid(which="minor", axis="y", linestyle=":", linewidth=0.5, color="black")
host.grid(which="minor", axis="x", linestyle=":", linewidth=0.5, color="black")

# Set legend
lines = [p1, p2]
labels = [line.get_label() for line in lines]
host.legend(lines, labels, loc="upper left", fontsize=12)

# Set colors
host.tick_params(axis="y", labelcolor="tab:blue")
par.tick_params(axis="y", labelcolor="tab:red")
host.yaxis.label.set_color("tab:blue")
par.yaxis.label.set_color("tab:red")

# Set title
plt.title(f"Temperature and Volume Loss - Step {TSTEP} days", fontsize=14)
plt.tight_layout()

plt.show()


In [ ]:
# Make plot for Cumulated volumes
major_locator = mdates.MonthLocator(interval=1)
major_formatter = mdates.ConciseDateFormatter(major_locator)
minor_locator =  mdates.WeekdayLocator(byweekday=0, interval=1)

fig, ax = plt.subplots()
fig.set_tight_layout(True)
p1 = ax.plot(df["date_in"], -df["volume_daily_norm_cumul"], label="Cumulated volume loss", linewidth=2, color="tab:blue")
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("-dV [$m^3$]", fontsize=12)
ax.set_title(f"Cumulated volume loss - Step {TSTEP} days", fontsize=12)
ax.grid(True)
ax.minorticks_on()
ax.grid(which="major", axis="y", linewidth=0.5, color="black")
ax.grid(which="major", axis="x", linewidth=0.3, color="black")
ax.grid(which="minor", axis="y", linestyle=":", linewidth=0.5, color="black")
ax.grid(which="minor", axis="x", linestyle=":", linewidth=0.3, color="black")
ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_formatter)
ax.xaxis.set_minor_locator(minor_locator)

# Set legend
lines = p1
labels = [line.get_label() for line in lines]
ax.legend(lines, labels, loc="upper left", fontsize=12)
plt.tight_layout()

# fig.set_size_inches(18.5, 10.5)
fig.savefig(out_dir / f"{fout_name}_daily_diff_norm_cumulated.png", dpi=300)

plt.show()

